In [ ]:
import multiModel as mm
from multiModel import MultiInputModel


import matplotlib.pyplot as plt

# Załaduj dane
test_dataset = mm.MultiInputDataset("CSV/dataset/test.csv", transform_rgb=mm.transform_rgb, transform_binary=mm.transform_binary)
test_loader = mm.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Wywołanie funkcji testującej
cm, y_true, y_pred = mm.test_model(
    path_model="training_results/best_model_mobilenet_v3_small.pth",
    test_loader=test_loader,
    device="cuda"
)

# Wyświetlenie macierzy pomyłek
class_names = list(test_dataset.class_to_idx.keys())
plt.figure(figsize=(10, 8))
mm.sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Przewidywane klasy")
plt.ylabel("Rzeczywiste klasy")
plt.title("Macierz pomyłek")
plt.show()

# Inne metryki
print("Dokładność:", mm.accuracy_score(y_true, y_pred))
print("\nRaport klasyfikacji:")
print(mm.classification_report(y_true, y_pred, target_names=class_names))


/media/3TB/github/Multi-input-Grain-CNN/multiModel.py:242: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(path_model, map_location=device)
